In [ ]:
%run '~/Zhang-Lab/Zhang Lab Code/Boilerplate/model_boilerplate.py'

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

est = xgb.XGBRFRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_estimators=100,
    max_depth=5,
    device='cuda',
    tree_method='hist'
)

model = MultiOutputRegressor(est)
model.fit(x_train, y_train, eval_set=[(x_val, y_val)])
predictions = model.predict(x_test)


In [ ]:
from sklearn.metrics import r2_score
import numpy as np

# Per‑target validation R²
print("Per‑target validation R² scores:")
val_pred = model.predict(x_val)          # shape: (n_samples, n_targets)

for i in range(y_val.shape[1]):
    val_r2 = r2_score(y_val[:, i], val_pred[:, i])
    print(f"Target {i}: {val_r2:.4f}")

# Aggregate test R²
test_pred = model.predict(x_test)
test_r2 = r2_score(y_test, test_pred)
print(f"\nAggregate test R²: {test_r2:.4f}")


In [ ]:
# saving model and metadata

joblib.dump(model, 'model_multioutput.joblib', compress=9)

versions = {
    'xgboost': xgb.__version__,
    'scikit-learn': __import__('sklearn').__version__,
    'joblib': joblib.__version__,
}
with open('model_versions.json', 'w') as f:
    json.dump(versions, f)